In [1]:
import pandas as pd
import sqlite3
import os
import time
import glob
# import counter
import matplotlib.pyplot as plt
from IPython.display import clear_output

# if not os.getcwd().endswith('/run'):
#     os.chdir('run')

RUN_FOLDER = '../data-books'
DBNAME = f"{RUN_FOLDER}/state.db"

In [ ]:
with sqlite3.connect(DBNAME) as conn:
    df = pd.read_sql("SELECT sid, attempt FROM attempts", conn)
# display(df.mean())
df.groupby('sid').count().mean().round(2)['attempt']

In [ ]:
with sqlite3.connect(DBNAME) as conn:
    df = pd.read_sql("SELECT sid, attempt, status, fetch_duration, total_duration FROM attempts WHERE status = 200 and total_duration > 0", conn)
df

In [ ]:
df.describe()

In [5]:
# with sqlite3.connect(DBNAME) as conn:
#     db_words_count = pd.read_sql("SELECT * FROM words ORDER BY count DESC, word ASC", conn)
# db_words_count[:10]

In [ ]:
with sqlite3.connect(DBNAME) as conn:
    df = pd.read_sql("SELECT * FROM pages", conn)

df = df.groupby('error')['sid'].count().to_frame().sort_values('sid', ascending=False).reset_index()
df = df.rename(columns={'sid':'count'})
print(df.to_string(index=False))


In [ ]:
with sqlite3.connect(DBNAME) as conn:
    df = pd.read_sql("SELECT * FROM pages", conn)
    db_words_count = pd.read_sql("SELECT * FROM words ORDER BY count DESC, word ASC", conn)

display(df.groupby('error')['sid'].count().to_frame().sort_values('sid', ascending=False))
display(df.groupby('status')['sid'].count().to_frame().sort_values('sid'))

print(f'pages files: {len(glob.glob(f'{RUN_FOLDER}/pages/**/*.*', recursive=True))}')
print(f' text files: {len(glob.glob(f'{RUN_FOLDER}/text/**/*.*', recursive=True))}')
print(f' word files: {len(glob.glob(f'{RUN_FOLDER}/words/**/*.*', recursive=True))}')

total_word_count, _ = counter.summ_counters_folder(f'{RUN_FOLDER}/words')
total_word_count_df = pd.DataFrame(total_word_count.items(), columns=['word','count']).sort_values(['count', 'word'], ascending=[False, True]).reset_index(drop=True)
display(total_word_count_df[:10])
print(f'Files word count identical to DB word count - {db_words_count.equals(total_word_count_df)}')

In [ ]:
def draw_progress_bar():
    clear_output(wait=True)
    
    with sqlite3.connect(DBNAME) as conn:
        df = pd.read_sql("SELECT * FROM pages", conn)

    # counts
    counts = df['status'].value_counts()
    visited = counts.get('visited', 0)
    failed = counts.get('failed', 0)
    queued = counts.get('queued', 0)
    done = visited + failed
    total = done + queued
    
    fig, ax = plt.subplots(figsize=(8, 1.5))
    ax.barh(0, visited, color='green', label=f'Visited: {visited} ({visited/total:.0%})')
    ax.barh(0, failed, left=visited, color='red', label=f'Failed : {failed} ({failed/total:.0%})')
    ax.barh(0, queued, left=done, color='orange', label=f'Queued : {queued} ({queued/total:.0%})')

    ax.set_title(f"Crawling Progress: {done} of {total} pages", fontsize=12, pad=5)
    ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5), ncol=3, fontsize=10, frameon=False)
    ax.set_xlim(0, total)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_frame_on(False)
 
    plt.subplots_adjust(bottom=0.3)
    plt.tight_layout()
    plt.show()
    return queued

_ = draw_progress_bar()

In [ ]:
while draw_progress_bar():
    time.sleep(1)